In [1]:
import psycopg2
import pandas as pd 

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="db-postgresql-sustainability-group-do-user-14262231-0.b.db.ondigitalocean.com",
    port=25060,
    database="defaultdb",
    user="doadmin",
    password="AVNS_xJlGYQTQBtMI5V_xcj1",
    sslmode="require"
)



In [2]:
industry_var = 'Financial Institution'
financial_institution_var = 'FALSE'
company_size_var = '>500 employees'
listed_on_regulated_markets_var = 'Yes'

In [3]:

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="db-postgresql-sustainability-group-do-user-14262231-0.b.db.ondigitalocean.com",
    port=25060,
    database="defaultdb",
    user="doadmin",
    password="AVNS_xJlGYQTQBtMI5V_xcj1",
    sslmode="require"
)

list_industries = ['Financial Institution', 'Insurance', 'Pension Funds', 'Auditor', 'Extractive', 'Industry', 'Large Infrastructure', 'Company']
list_industries_sfdr = ['Investment firms', 'pension funds', 'asset managers', 'insurance companies', 'banks', 'venture capital funds', 'credit institutions offering portfolio management', 'financial advisors']
df_directives = pd.DataFrame(columns=['directive', 'start_date'])
start_of_rep = ''

cur = conn.cursor()

# query the csrd description 
query_csrd_name = "SELECT csrd_name FROM csrd"
query_csrd_description = "SELECT csrd_description FROM csrd"



# check for csrd
if company_size_var == '>500 employees' and (industry_var in list_industries or listed_on_regulated_markets_var == 'Yes'):
    df_directives.loc[len(df_directives)] = ['CSRD', '2025 (for 2024)']

elif (company_size_var == '>250 employees' and total_asset_var == '>20Mio') or (company_size_var == '>250 employees' and net_revenue_var == '>40Mio') or (total_asset_var == '>20Mio' and net_revenue_var == '>40Mio'): 
    df_directives.loc[len(df_directives)] = ['CSRD', '2026 (for 2025)']
    
elif (company_size_var == '<250 employees' and total_asset_var == '<20Mio' and listed_on_regulated_markets_var == 'Yes'):
    df_directives.loc[len(df_directives)] = ['CSRD', '2026 (for 2025)']

else:
    pass


# nfdr
if company_size_var == '>500 employees' and (industry_var in list_industries or listed_on_regulated_markets_var == 'Yes'):
    df_directives.loc[len(df_directives)] = ['NFRD', '2017 until 2025']


# EU-Taxonomy 
if company_size_var == '>500 employees' and (industry_var in list_industries or listed_on_regulated_markets_var == 'Yes'):
    df_directives.loc[len(df_directives)] = ['EU-Taxonomy', '2022']

# sfdr 
if industry_var in list_industries_sfdr:
    df_directives.loc[len(df_directives)] = ['SFDR Level 1', '2021']
    df_directives.loc[len(df_directives)] = ['SFDR Level 2', '2023']



result_df = pd.DataFrame(columns=['directive', 'start_date',  'csrd_name', 'csrd_description' ])


for index, row in df_directives.iterrows():

    directive = row['directive']
    start_date = row['start_date']


    if directive == 'CSRD':
        # Query the "csrd" table
        cur.execute("SELECT csrd_name FROM csrd")
        csrd_names = str(cur.fetchall())

        cur.execute("SELECT csrd_description FROM csrd")
        csrd_description = str(cur.fetchall())

        result_df.loc[len(result_df)] = ['CSRD', start_date, csrd_names, csrd_description]


    elif directive == 'NFRD':
        # Query the "nfrd" table
        cur.execute("SELECT nfrd_name FROM nfrd")
        nfrd_names = str(cur.fetchall())

        cur.execute("SELECT nfrd_description FROM nfrd")
        nfrd_description = str(cur.fetchall())

        result_df.loc[len(result_df)] = ['NFRD', start_date, nfrd_names, '']

    elif directive == 'EU-Taxonomy':
        # Query the "Eu-Taxonomy" table
        cur.execute("SELECT eu_taxonomy_name FROM eu_taxonomy")
        eu_taxonomy_names = str(cur.fetchall())

        cur.execute("SELECT eu_taxonomy_description FROM eu_taxonomy")
        eu_taxonomy_description = str(cur.fetchall())

        result_df.loc[len(result_df)] = ['EU-Taxonomy', start_date, eu_taxonomy_names, eu_taxonomy_description]

    elif directive == 'SFDR Level 1':
        # Query the "Eu-Taxonomy" table
        cur.execute("SELECT sfdr_name FROM sfdr WHERE sfdr_name LIKE 'Level 1%'")
        sfdr_names = str(cur.fetchall())

        cur.execute("SELECT sfdr_description FROM sfdr WHERE sfdr_name LIKE 'Level 1%'")
        sfdr_description = str(cur.fetchall())

        result_df.loc[len(result_df)] = ['SFDR Level 1', start_date, sfdr_names, '']

    elif directive == 'SFDR Level 2':
        # Query the "Eu-Taxonomy" table
        cur.execute("SELECT sfdr_name FROM sfdr WHERE sfdr_name LIKE 'Level 2%'")
        sfdr_names = str(cur.fetchall())

        cur.execute("SELECT sfdr_description FROM sfdr WHERE sfdr_name LIKE 'Level 2%'")
        sfdr_description = str(cur.fetchall())

        result_df.loc[len(result_df)] = ['SFDR Level 2', start_date, sfdr_names, sfdr_description]


    else: 
        break

# Close the cursor and connection
cur.close()
conn.close()

In [5]:
for index, row in result_df.iterrows():

    directive = row['csrd_name']
    start_date = row['csrd_description']




In [7]:
start_date

"[('',), ('Broken down by taxonomy compliant economic activity, indicating which environmental objective this activity contributes to and whether it is a transitional or enabling economic activity',), ('Broken down by taxonomy compliant economic activity, indicating which environmental objective this activity contributes to and whether it is a transitional or enabling economic activity',), ('Criteria criteria for determining the conditions under which an economic activity qualifies as contributing substantially to climate change mitigation or climate change adaptation and for determining whether that economic activity causes no significant harm to any of the other environmental objectives.',), ('',)]"

In [4]:
result_df

,directive,start_date,csrd_name,csrd_description
0,CSRD,2025 (for 2024),"[('General principles',), ('Double materiality...","[('General, strategy, governance and materiali..."
1,NFRD,2017 until 2025,"[('Environmental issues',), ('Social issues an...",
2,EU-Taxonomy,2022,[('Environmentally sustainable portion of the ...,"[('',), ('Broken down by taxonomy compliant ec..."
